In [ ]:
import pythia8
import numpy as np
from tqdm.notebook import tqdm
import ICARUSservices as serv
import itertools
from multiprocessing import Pool
import multiprocessing as mp

In [ ]:
dosave = True
prefix = "./numi_icarus_incident_"

meson_per_file = int(1e6)

In [ ]:
# Detector config -- we're only going to generate stuff that points at ICARUS
geo = serv.ServiceManager("Geometry")

ICARUSDET = np.array([
    min([geo.TPC(t, c).ActiveBoundingBox().MinX() for t in range(4) for c in range(2)]) - 10,
    max([geo.TPC(t, c).ActiveBoundingBox().MaxX() for t in range(4) for c in range(2)]) + 10,
    
    min([geo.TPC(t, c).ActiveBoundingBox().MinY() for t in range(4) for c in range(2)]) - 10,
    max([geo.TPC(t, c).ActiveBoundingBox().MaxY() for t in range(4) for c in range(2)]) + 10,
    
    min([geo.TPC(t, c).ActiveBoundingBox().MinZ() for t in range(4) for c in range(2)]) - 10,
    max([geo.TPC(t, c).ActiveBoundingBox().MaxZ() for t in range(4) for c in range(2)]) + 10,
])

# Taken from: https://github.com/SBNSoftware/icaruscode/blob/develop/fcl/gen/numi/GNuMIFlux.xml#L765
beam2detector = np.array([[0.921035925, 0.022715103,  0.388814672],
                          [0, 0.998297825, -0.058321970],
                          [-0.389477631, 0.053716629,  0.919468161]])

numi_beam_origin = [0, 0, 0, 4.503730e2, 80.153901e2, 795.112945e2]

detcoord_beam_origin = np.array(numi_beam_origin[:3]) - np.matmul(beam2detector, numi_beam_origin[3:])

phis = []
for corner in map(np.array, itertools.product(ICARUSDET[:2], ICARUSDET[2:4], ICARUSDET[4:])):
    beam_coord_origin2corner = np.matmul(beam2detector.T, corner - detcoord_beam_origin)    
    phi = np.arctan2(beam_coord_origin2corner[1], beam_coord_origin2corner[0])
    
    phis.append(phi)
    
philo = min(phis)
phihi = max(phis)

In [ ]:
detcoord_beam_origin

In [ ]:
(phihi - philo) / (2*np.pi)

In [ ]:
def intersects_detector(x, v):
    normals = [np.array([1, 0, 0]),
               -np.array([1, 0, 0]),
               np.array([0, 1, 0]),
               -np.array([0, 1, 0]),
               np.array([0, 0, 1]),
               -np.array([0, 0, 1]),
              ]
    
    normal_offsets = [
        np.array([ICARUSDET[0], ICARUSDET[2], ICARUSDET[4]]),
        np.array([ICARUSDET[1], ICARUSDET[2], ICARUSDET[4]]),
        np.array([ICARUSDET[0], ICARUSDET[2], ICARUSDET[4]]),
        np.array([ICARUSDET[0], ICARUSDET[3], ICARUSDET[4]]),
        np.array([ICARUSDET[0], ICARUSDET[2], ICARUSDET[4]]),
        np.array([ICARUSDET[0], ICARUSDET[2], ICARUSDET[5]])
    ]
    ips = []
    for i, (n, o) in enumerate(zip(normals, normal_offsets)):
        dim = i // 2
        
        # check for orthogonal
        if np.sum(n*v) == 0.: continue
        
        lp = np.sum(n*(o - x)) / np.sum(n*v)        
        ip = x + lp*v
        
        for d in range(3):
            if d == dim: continue
                                
            if ip[d] > ICARUSDET[2*d+1] or ip[d] < ICARUSDET[2*d]:
                break
        else:
            ips.append(ip)
                        
    return len(ips) == 2

def intersects_detector_beamcoord(x, v):
    x = np.matmul(beam2detector, x) + detcoord_beam_origin
    v = np.matmul(beam2detector, v)
    
    return intersects_detector(x, v)

In [ ]:
th = 5.7*np.pi/180
phi = philo + 0.01
intersects_detector_beamcoord(np.array([0, 0, 0]), 
                              np.array([np.sin(th)*np.cos(phi), np.sin(th)*np.sin(phi), np.cos(th)]))

In [ ]:
def init_pythia():
    global pythiap
    global pythian
    
    ID = int(mp.current_process().name.split('-')[1])    
    # pp event generator
    pythiap = pythia8.Pythia()
    pythiap.readString("Main:numberOfEvents = " + str(nevents))
    # Fixed seed for now
    pythiap.readString("Random:setSeed = on")
    pythiap.readString("Random:seed = %i" % (1234+ID))
    # Proton on proton
    pythiap.readString("Beams:idA = 2212")
    pythiap.readString("Beams:idB = 2212")
    # Do a safer CM frame generation (120 GeV kinetic lab energy = 15.123 GeV CM)
    # Weird things happen for me in lab frame
    pythiap.readString("Beams:eCM = 15.123")
    # The consensus right process seems to be SoftQCD, but can consider others
    #pythiap.readString("HardQCD:hardccbar = on")
    #pythiap.readString("LowEnergyQCD:all = on")
    pythiap.readString("SoftQCD:all = on")
    # Don't put any phase space cut "by hand"
    pythiap.readString("PhaseSpace:mHatMin = 0.")

    # pn event generator
    # Settings are largely the same, but use n instead of p
    pythian = pythia8.Pythia()
    pythian.readString("Main:numberOfEvents = " + str(nevents))
    pythian.readString("Random:setSeed = on")
    pythian.readString("Random:seed = %i" % (ID + 5678))
    pythian.readString("Beams:idA = 2212")
    pythian.readString("Beams:idB = 2112")
    pythian.readString("Beams:eCM = 15.133")
    #pythian.readString("HardQCD:hardccbar = on")
    #pythian.readString("LowEnergyQCD:all = on")
    pythian.readString("SoftQCD:all = on")
    pythian.readString("PhaseSpace:mHatMin = 0.")
    # don't decay the tracked particles
    for tp in tracked_particles:
        pythiap.readString(str(tp) + ":mayDecay = off")
        pythian.readString(str(tp) + ":mayDecay = off")

    pythiap.init()
    pythian.init()
    
def run_pythia(_):
    evt = None
    # decide to do pp or pn
    if np.random.random() < float(Z)/A:
        if not pythiap.next():
            return []
        evt = pythiap.event
    else:
        if not pythian.next():
            return []
        evt = pythian.event
    # Use only if doing CM frame collisions
    pbeam = pythia8.Vec4(0,0,0,0)

    # Find the particles we want to save
    entries = []
    for part in evt:
        # Find the beam proton
        # event generation the CM frame: need to boost to lab frame
        if part.idAbs() == 2212 and part.status() == -12 and part.pz() > 0:
            pbeam = part.p()
        # Skip non-final
        if not part.isFinal():
            continue
        # Make sure we found the beam
        if pbeam.e() == 0.:
            raise ValueError('Beam energy vector has not been set.')

        if any([part.idAbs() == pdg for pdg in tracked_particles]):
            pdg = part.idAbs()
            # do the prescale
            prescale = prescales[pdg]
            if np.random.random() > 1. / (prescale): continue

            wgt = prescale / NPHI
            if FORCE_PHI:
                wgt = wgt * (phihi - philo) / (2*np.pi)

            plab = part.p()
            plab.bst(pbeam)
            px = plab.px()
            py = plab.py()
            pz = plab.pz()
            e  = plab.e()
            
            costh = pz / np.sqrt(px**2 + py**2 + pz**2)
            sinth = np.sqrt(1 - costh**2)
            for _ in range(NPHI):
                # Force PHI if we want to
                if FORCE_PHI:
                    phi = np.random.uniform(low=philo, high=phihi)
                    px = pz*sinth*np.cos(phi)
                    py = pz*sinth*np.sin(phi)
                    
                entry = [px, py, pz, e, pdg, wgt*WGTSCALE]
                x = np.array([0, 0, 0])
                v = np.array([px, py, pz]) / np.sqrt(px**2 + py**2 + pz**2)
                if REQUIRE_HIT_DETECTOR and not intersects_detector_beamcoord(x, v):
                    continue
                                        
                entries.append(entry)

    return entries

# Meson decay generation

In [ ]:
nevents = int(5e7)  # number of POT to generate

Z = 6          # Z for carbon (graphite target @ NuMI)
A = 12         # A for carbon

# Particles to store and files to store them in
tracked_particles = [111, 221, 331]

files = {
    'pion': [111],
    'eta': [221],
    'eta_prime': [331],
    'psuedoscalars': [111, 221, 331],
}

prescales = {
    111: 100.,
    221: 1.,
    331: 1.
}

# Only 87% of protons interact in the Arget
WGTSCALE = 0.87

# Do we make the meson-ray hit the detector?
REQUIRE_HIT_DETECTOR = True

# Whether to force PHI into phlim
FORCE_PHI = True

# How many Phi-tries to make for each eta?
NPHI = 5

In [ ]:
# progress bar
pbar = tqdm(total=nevents)
    
tosave = {}
npots = {}
for f in files.keys():
    tosave[f] = []
    npots[f] = 0
        
# loop in pool
all_entries = []

with Pool(20, initializer=init_pythia) as p:
    for entries in p.imap_unordered(run_pythia, range(nevents)):
        # update progress bar
        pbar.update(1)
        
        for f in files:
            npots[f] += 1

        if len(entries) == 0:
            continue

        # allocate them to the different outputs
        for f in tosave.keys():
            this_entries = [e for e in entries if e[-2] in files[f]]
            npot = npots[f]
            for e in this_entries:
                tosave[f].append(e + [float(npot) / len(this_entries)])

            if len(this_entries) > 0: # reset POT counter
                npots[f] = 0     
    
pbar.close()

In [ ]:
for fname in files.keys():
    this_entries = tosave[fname]
    nchunk = 0
    ichunk = 0
    while nchunk < len(this_entries):
        chunked_entries = this_entries[nchunk:nchunk+meson_per_file]
        filename = prefix + fname + str(ichunk) + ".txt"
        with open(filename, "w") as f:
            f.write('# px\tpy\tpz\te\tpdgcode\tweight\tnpot\n')
            for e in chunked_entries:
                f.write("\t".join(map(str,e)) + "\n")
                
        ichunk += 1
        nchunk += meson_per_file

# Tau generation

## Here I am attempting to get decays of taus into HNLs

In [ ]:
nevents = int(1e4)
Z = 6
A = 12

# Particles to store and files to store them in
tracked_particles = [15]

files = {
    'tau': [15]
}



In [ ]:
# pp event generator
pythiap = pythia8.Pythia()
pythiap.readString("Main:numberOfEvents = " + str(nevents))
pythiap.readString("Random:setSeed = on")
pythiap.readString("Random:seed = 1234")
pythiap.readString("Beams:idA = 2212")
pythiap.readString("Beams:idB = 2212")
pythiap.readString("Beams:eCM = 15.123")
pythiap.readString("SoftQCD:all = on")
# In principle, hard ccbar seems better, but can't be mixed and both give D
#pythiap.readString("HardQCD:hardccbar = on")
# First turn off all SM tau decays
pythiap.readString("15:offIfAny = 16")
# Turn on decays
pythiap.readString("15:mayDecay = on")
# Decay tau in a 2 body channel with a final state sterile neutrino and charged pion 100% of the time
pythiap.readString("15:addChannel = 1 1.0 1521 18 -211")
pythiap.readString("18:m0 = 0.4")
pythiap.readString("18:mayDecay = off")
# Force the D and D_s to decay to tau
pythiap.readString("411:onMode = off")
pythiap.readString("411:onIfAny = 15")
pythiap.readString("431:onMode = off")
pythiap.readString("431:onIfAny = 15")
pythiap.readString("PhaseSpace:mHatMin = 0.")

# pn event generator
pythian = pythia8.Pythia()
pythian.readString("Main:numberOfEvents = " + str(nevents))
pythian.readString("Random:setSeed = on")
pythian.readString("Random:seed = 5678")
pythian.readString("Beams:idA = 2212")
pythian.readString("Beams:idB = 2112")
pythian.readString("Beams:eCM = 15.133")
pythian.readString("SoftQCD:all = on")
pythian.readString("15:offIfAny = 16")
pythian.readString("15:mayDecay = on")
pythian.readString("15:addChannel = 1 1.0 1521 18 -211")
pythian.readString("18:m0 = 0.4")
pythian.readString("18:mayDecay = off")
# Enhance decays to tau, but should be careful to weight appropriately
pythian.readString("411:onMode = off")
pythian.readString("411:onIfAny = 15")
pythian.readString("431:onMode = off")
pythian.readString("431:onIfAny = 15")
pythian.readString("PhaseSpace:mHatMin = 0.")

# don't decay the tracked particles
outfilename = 'hnl_new.txt'
with open(outfilename,'w') as f:
    f.write('# px\tpy\tpz\te\n')
# initialize and loop
pythiap.init()
pythian.init()
nD = 0
nDs = 0
for i in range(nevents):
    evt = None
    if np.random.random() < float(Z)/A:
        if not pythiap.next():
            continue
        evt = pythiap.event
    else:
        if not pythian.next():
            continue
        evt = pythian.event
    pbeam = pythia8.Vec4(0,0,0,0)
    for part in evt:
        # Find the beam proton
        # event generation the CM frame: need to boost to lab frame
        if part.idAbs() == 2212 and part.status() == -12 and part.pz() > 0:
            pbeam = part.p()
        if part.idAbs() == 411:
            nD += 1
        if part.idAbs() == 431:
            nDs += 1
        if not part.isFinal():
            continue
        # Make sure we found the beam
        if pbeam.e() == 0.:
            raise ValueError('Beam energy vector has not been set.')
        if part.idAbs() == 18:
            plab = part.p()
            plab.bst(pbeam)
            px = plab.px()
            py = plab.py()
            pz = plab.pz()
            e = plab.e()
            with open(outfilename,'a') as f:
                f.write(str(px) + '\t' + str(py) + '\t' + str(pz) + '\t' + str(e) + '\n')

#### 